# Pantano de códigos

- Local pra armazenar as tentativas com python, pandas e spark;


## Tasks do job

1. Ingestão de dados

    consumo_api.py
    
- Entrada: endpoint
- Transformação: agregação por coluna
- Saída: dados_api.json

2. Enviar os dados para o S3
    
    ingestao_batch.py
    
- Entrada: dados_api.json
- Saída: URI do S3 (s3://dados-num-lago/Raw/TMDB/JSON)

In [ ]:
## Lista dos gêneros 
### de filmes
#endpoint = "3/genre/movie/list"

### Armazenamento de dados em um DataFrame

#### Migrando o conteúdo do arquivo de texto para um RDD

In [ ]:
# Ler o arquivo | SparkContext (RDD)
linhas = sc.textFile(caminho_texto)
#linhas.collect()
# Limpeza das linhas
palavras = linhas.flatMap(lambda linha: linha.strip().lower.split())
# Contagem das palavras
contagem = palavras.map(lambda palavra: (palavra, 1))
# Redução das quantidades
reduzido = contagem.reduceByKey(lambda a,b: a + b)
dados = reduzido
colunas = ["Palavra", "Quantidade"]
df = spark.createDataFrame(dados, colunas)

#### Armazenamento de dados em um DataFrame | Pandas

In [ ]:
# Criar um DataFrame do Spark
colunas = ["Palavra", "Quantidade"]
df = spark.createDataFrame(mapeamento_palavras, colunas)


### Transformações | Job 3

In [ ]:
# Importe as bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType
import re
from unidecode import unidecode

# Inicie uma sessão Spark
spark = SparkSession.builder.appName("ProcessarArquivo").getOrCreate()

# Defina uma função para processar as linhas e criar um dicionário
def processar_linha(linha):
    mapeamento = {}
    for palavra in [
        re.sub(r'[^a-z]', '', unidecode(w.lower()))
        for w in re.findall(r'\b\w+\b', unidecode(linha.lower()))
    ]:
        if palavra:
            mapeamento[palavra] = mapeamento.get(palavra, 0) + 1
    return mapeamento

# Registre a função como uma função UDF (User-Defined Function)
schema = StructType([StructField("linha", StringType(), True)])
processar_linha_udf = udf(processar_linha, schema)

# Carregue o arquivo de texto em um DataFrame
caminho_texto = "./README.md"  # Substitua pelo caminho do seu arquivo
df = spark.read.text(caminho_texto)

# Aplique a função UDF para processar as linhas e criar uma coluna com o resultado
df_com_mapeamento = df.withColumn("mapeamento", processar_linha_udf(df["value"]))

# Colete o DataFrame em uma lista de dicionários
lista_de_dicionarios = df_com_mapeamento.select("mapeamento").rdd.flatMap(lambda x: x).collect()

# Crie um dicionário final combinando todos os dicionários na lista
dicionario_final = {}
for dicionario in lista_de_dicionarios:
    for chave, valor in dicionario.items():
        dicionario_final[chave] = dicionario_final.get(chave, 0) + valor

# Exiba o dicionário final
print(dicionario_final)

# Encerre a sessão Spark
spark.stop()


### Funções e Classes

In [6]:
# import sys

class WordMapper:
    def __init__(self):
        pass

    def map(self, line):
        words = line.split()
        for word in words:
            print(f"{word}\t1")

# Exemplo de uso
mapper = WordMapper()
for line in sys.stdin:
    mapper.map(line)

#### Limpeza das linhas

In [8]:
def processar_linha(linha):
    mapeamento = {}
    for palavra in [
        re.sub(r'[^a-z]', '', unidecode(w.lower()))
        for w in re.findall(r'\b\w+\b', unidecode(linha.lower()))
    ]:
        if palavra:
            mapeamento[palavra] = mapeamento.get(palavra, 0) + 1
    return mapeamento

In [5]:
def ler_arquivo(caminho):
    try:
        with open(caminho, "r") as arquivo:
            return list(enumerate(map(str.strip, arquivo), start=1))
    except FileNotFoundError:
        return []


## Tarefa do Job | Conta palavras

**Objetivo:** contar a quantidade de ocorrências das palavras em um arquivo de texto.

## Passos

1. Escrever uma função para mapear as palavras: WordMapperClass -> wordMapper(); 
2. Escrever uma função para reduzir a lista de frequência: WordReducerClass -> wordReducer();
3. Escrever um script que aponte para as funções criadas e execute os métodos: WordCounterClass -> main();

In [2]:
import os 
import sys

os.environ["PYSPARK_SESSION"] = sys.executable

# Spark
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
spark = SparkSession.builder.master("local[*]").appName('SparkHelloWord').getOrCreate()

### Passo 1: WordMapper Class

- A função irá rodar uma só vez por cada linha do arquivo de texto; 
- **Input:** {"num_da_linha": "Texto da linha"}
- **Output:** {"palavra": "quantidade"}

In [ ]:
def WordMapper(args):
    
    for line in sys.stdin:
        for word in line.split():
            print "%s\t%s" % (word,1)
    
    return par_palavras

## Passo 2: WordReducer Class

- A função irá rodar uma só vez por cada par de chave-valor ordenado pelo Hadoop;

- **Input:** {"palavra": "lista_de_frequencia"}
- **Output:** {"palavra": "contagem"}

In [ ]:
def WordReducer(args):
    
    (last_key,count) = (None, 0)
    for line in sys.stdin:
        (key, value) = line.strip().split("\t")
        if last_key and last_key!=key:
            print "%s\t%s" % (last_key,count)
            (last_key,count) = (key,int(value))
        else:
            (last_key,count) = (key, count + int(value))
    
    return 

## Passo 3: Main

- A função principal do programa contendo todas as configurações necessárias para realizar a tarefa;

In [ ]:
import sys

def WordCounter(input_path, output_path):
    return


**Objetivo:** contar a quantidade de ocorrências das palavras em um arquivo de texto.

## Passos

1. Escrever uma função para mapear as palavras: WordMapperClass -> wordMapper(); 
2. Escrever uma função para reduzir a lista de frequência: WordReducerClass -> wordReducer();
3. Escrever um script que aponte para as funções criadas e execute os métodos: WordCounterClass -> main();

In [2]:
import os 
import sys

os.environ["PYSPARK_SESSION"] = sys.executable

# Spark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName('SparkHelloWord').getOrCreate()

### Passo 1: WordMapper Class

- A função irá rodar uma só vez por cada linha do arquivo de texto; 
- **Input:** {"num_da_linha": "Texto da linha"}
- **Output:** {"palavra": "quantidade"}

In [ ]:
def WordMapper(args):
    
    for line in sys.stdin:
        for word in line.split():
            print "%s\t%s" % (word,1)
    
    return par_palavras

## Passo 2: WordReducer Class

- A função irá rodar uma só vez por cada par de chave-valor ordenado pelo Hadoop;

- **Input:** {"palavra": "lista_de_frequencia"}
- **Output:** {"palavra": "contagem"}

In [ ]:
def WordReducer(args):
    
    (last_key,count) = (None, 0)
    for line in sys.stdin:
        (key, value) = line.strip().split("\t")
        if last_key and last_key!=key:
            print "%s\t%s" % (last_key,count)
            (last_key,count) = (key,int(value))
        else:
            (last_key,count) = (key, count + int(value))
    
    return 

## Passo 3: Main

- A função principal do programa contendo todas as configurações necessárias para realizar a tarefa;

In [ ]:
import sys

def WordCounter(input_path, output_path):
    return


## Sprint 6 | Athena

- Crie uma consulta que lista os 3 nomes mais usados em cada década desde o 1950 até hoje.

In [ ]:
# Extrair o ano da coluna 'ano'
df['ano'] = pd.to_numeric(df['ano'], errors='coerce')

# Encontrar o intervalo de décadas com base nos anos presentes no DataFrame
min_year = df['ano'].min()
max_year = df['ano'].max()
decadas_validas = range(min_year // 10 * 10, (max_year // 10 + 1) * 10, 10)

# Filtrar o DataFrame para incluir apenas as décadas válidas
df = df[df['ano'].isin(decadas_validas)]

# Definir uma função para extrair a década a partir do ano
def extrair_decada(ano):
    return ano // 10 * 10

# Nova coluna chamada década
df['decada'] = df['ano'].apply(extrair_decada)

# Agrupar por década e contar os nomes mais comuns em cada grupo
resultado = df.groupby(['decada', 'nome'])['total'].sum().reset_index()

# Ordenar o resultado pela decada e pelo total
resultado = resultado.sort_values(by=['decada', 'total'], ascending=[True, False])

# Listar os 3 nomes mais usados em cada década
decadas = resultado['decada'].unique()

for decada in decadas:
  if decada >= 1950:
    top_nomes_decada = resultado[resultado['decada'] == decada].head(3)
    print(f"Década: {decada}")
    print(top_nomes_decada[['nome', 'total']])
    print()

UsageError: Line magic function `%help` not found.
